# DELPHI simulations on Resistive Wall + Resonator
## Looking at eigenvalues and eigenvectors output by DELPHI

### 0.1) Import libraries

In [1]:
%pylab notebook

from __future__ import division
from Impedance import *
from DELPHI import *
from string import *
from LHC_param import LHC_param
from SPS_param import SPS_param
import itertools
import os
from  scipy.constants import c as clight
import scipy.special
from scipy.misc import factorial
import collections
#import skrf as rf


#from matplotlib import rc
#rc('font',**{'family':'serif','serif':['Times New Roman'],'size':12})
#rc('text', usetex=True)

Populating the interactive namespace from numpy and matplotlib


/afs/cern.ch/user/d/damorim/work/GitIRIS/IW2D/PYTHON_codes_and_scripts/Impedance_lib_Python/Impedance.py:1299: SyntaxWarning: import * only allowed at module level
  def plot_impmod(imp_mod,comp,axtypex,axtypey,nfig):


### 0.2) Machine parameters

In [2]:
#Particle parameters (from LHC)
energy = 450.0 #in GeV
e,m0,c,E0 = proton_param()
machine,E,gamma,sigmaz,taub,R,Qx,Qxfrac,Qy,Qyfrac,Qs,eta,f0,omega0,omegas,dphase,Estr,V,h = LHC_param(E0,E=energy*1e9);

f0 = omega0/(2*np.pi)
fs = omegas/(2*np.pi)

print 'Qx = %f , Qxfrac = %f' % (Qx, Qxfrac)
print 'Qs = %f' % (Qs)
print 'omega0 = %f , omegas = %f' % (omega0, omegas)
print 'f0 = %f , fs = %f' % (omega0/(2.*np.pi), omegas/(2.*np.pi))

Qx = 64.310000 , Qxfrac = 0.310000
Qs = 0.004905
omega0 = 70657.404242 , omegas = 346.596859
f0 = 11245.475151 , fs = 55.162603


## 1) Resistive Wall + Broadband Resonnator Impedance Model

In [4]:
#Construction of the chamber
thick = 5e-3
    
Cu_lay = Cu300K_layer(thickness=thick)
vac_lay = vacuum_layer(thickness=np.inf)

chamber = construct_layers([Cu_lay,vac_lay])


#Frequency range
freq = freq_param(fmin=1e2,fmax=1e10,ftypescan=2,nflog=20,fadded=[],fmaxrefine=1e3,fminrefine=1e10,nrefine=500)


#Executables paths
#IW2D executables path
IW2Dexec='/afs/cern.ch/user/d/damorim/work/GitIRIS/IW2D/ImpedanceWake2D'

#IW2D result files location and names
ResultsDirIW2D='/afs/cern.ch/user/d/damorim/work/IW2D/RW_Resonator_round_chamber/'
comment='_Thick'+str(thick*1e3)+'mm'



#Resistive wall Impedances simulations
imp_model_RW_R = []; wake_model_RW_R = []

iw_input_R = impedance_wake_input(machine='LHC',b=[10e-3],layers=chamber,fpar=freq,geometry='round',comment=comment)
imp_model_RW_R, wake_model_RW_R = imp_model_from_IW2D(iw_input_R,wake_calc=False,path=IW2Dexec,flagrm=True, lxplusbatch=None,dire=ResultsDirIW2D);



#Resonator model
imp_resonator = []; wake_resonator = []

#For info: Function header:
#imp_model_resonator(Rlist,frlist,Qlist,beta=1,wake_calc=False,fpar=freq_param(ftypescan=0,nflog=100),zpar=z_param(),listcomp=['Zxdip','Zydip']):

imp_resonator, wake_resonator = imp_model_resonator(np.array([1.0e3]),np.array([1.0e9]),np.array([1]),beta=1,wake_calc=False, fpar=freq,zpar=z_param(),listcomp=['Zxdip'])



#Total impedance calculation: RW + resonator
imp_total_RW_R = add_impedance_wake(imp_model_RW_R,imp_resonator,weightx=1,weighty=1)


#Writing impedance files
write_imp_wake_mod(imp_total_RW_R,name='RW_Resonator',listcomp=['Zl','Zxdip','Zydip','Zxquad','Zyquad'],dire=ResultsDirIW2D)

## 2) Computation with eigenmodesDELPHI_converged


### 2.1) Parameters for eigenmodesDELPHI_converged

In [5]:
#DELPHI parameters

#Number of bunches, coupled bunch mode, particles per bunch
M = 1 # 2748, 3564
nx = 0
Nb = 1e11

#Distribution
g,a,b=longdistribution_decomp(taub,typelong="Gaussian");

#Chromaticity
Qp = 0
omegaksi = Qp*omega0/eta

#Frequency downsampling
nevery = 1 

#Convergence criterion
crit = 5.e-3
abseps = 1.e-3



#Convert IW2D output into lists for DELPHI
#Here we take the Zxdip component
Zlist_RW_R = deepcopy(imp_total_RW_R[1].func[::nevery,:])
freqlist_RW_R = deepcopy(imp_total_RW_R[1].var[::nevery])

# Impedance and damper matrix coefficient computation
coefdamper_RW_R,coefZ_RW_R = computes_coef(f0,0,b,g[0],1,taub,0,M,Nb,gamma,Qx,particle='proton');

#Tables used for eigenmodesDELPHI output
lmaxold = 1
nmaxold = 1

eigenvalues_converged_RW_R = np.zeros(((2*lmaxold+1)*(nmaxold+1)), dtype = complex)
eigenvectors_converged_RW_R = np.zeros(((2*lmaxold+1)*(nmaxold+1),(2*lmaxold+1)*(nmaxold+1)), dtype = complex)

matdamperold = np.zeros((2*lmaxold+1,nmaxold+1,2*lmaxold+1,nmaxold+1),dtype=complex)
matZold = np.zeros((2*lmaxold+1,nmaxold+1,2*lmaxold+1,nmaxold+1),dtype=complex)

### 2.2) Computation with eigenmodesDELPHI_converged

In [6]:
#Function header
#eigenmodesDELPHI_converged(nx,M,omegaksi,omega0,tunefrac,a,b,taub,g,Z,freqZ,coefdamper,coefZ,omegas,flag_trapz=0,flagdamperimp=0,d=None,freqd=None,kmax=5,crit=5.e-2,abseps=1.e-3,lmaxold=-1,nmaxold=-1,matdamperold=None,matZold=None,flageigenvect=False)

#Computation with eigenmodesDELPHI_converged
eigenvalues_converged_RW_R, eigenvectors_converged_RW_R, lmax0, nmax0, matdamperold, matZold = eigenmodesDELPHI_converged(nx,M,omegaksi,omega0,Qxfrac,a,b,taub,g,Zlist_RW_R,freqlist_RW_R,coefdamper_RW_R,coefZ_RW_R,omegas,flag_trapz=1,flagdamperimp=0,d=None,freqd=None,kmax=1,crit=crit,abseps=abseps,lmaxold=-1,nmaxold=-1,matdamperold=None,matZold=None,flageigenvect=True)


lmax= 1 , nmax= 1 , time for computing matrices [seconds]:  0.15
lmax= 1 , nmax= 1 , time for eigenvalue pb resolution [seconds]:  0.01
lmax= 2 , nmax= 2 , time for computing matrices [seconds]:  0.05
lmax= 2 , nmax= 2 , time for eigenvalue pb resolution [seconds]:  0.0


/afs/cern.ch/user/d/damorim/scratch0/IRIS/PYTHON_codes_and_scripts/DELPHI_Python/DELPHI.py:171: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (damperold==None) or ( ( (l>lmaxold)or(lprime>lmaxold) ) or ( (n>nmaxold)or(nprime>nmaxold) ) ):
/afs/cern.ch/user/d/damorim/scratch0/IRIS/PYTHON_codes_and_scripts/DELPHI_Python/DELPHI.py:81: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (couplold==None) or ( ( (l>lmaxold)or(lprime>lmaxold) ) or ( (n>nmaxold)or(nprime>nmaxold) ) ):


In [7]:
#The eigenvalues are ordered by their imaginary part: the most unstable mode is first

print eigenvalues_converged_RW_R

[  3.46593671e+02 -9.84554954e-15j  -3.46600047e+02 -2.90021186e-15j
  -3.46597043e+02 -4.75310130e-16j   3.46596676e+02 -4.58990416e-16j
   6.93193765e+02 -2.30301356e-16j   6.93193701e+02 -1.59680919e-16j
  -3.46596837e+02 -1.28409239e-17j   3.46596882e+02 -5.53918511e-18j
  -6.93193672e+02 +5.33882196e-19j  -6.93193737e+02 +6.21791885e-19j
   6.93192819e+02 +3.59389916e-17j  -6.93194619e+02 +3.59652467e-17j
  -1.46247055e-05 +4.36889672e-09j  -1.13673102e-03 +1.64680470e-07j
  -1.56195973e-02 +4.13702704e-06j]


In [8]:
#The tuneshifts are ordered by their imaginary part: the most unstable mode is first

print eigenvalues_converged_RW_R/omega0

[  4.90527037e-03 -1.39342078e-19j  -4.90536061e-03 -4.10461139e-20j
  -4.90531809e-03 -6.72696846e-21j   4.90531289e-03 -6.49599884e-21j
   9.81063163e-03 -3.25940866e-21j   9.81063072e-03 -2.25993186e-21j
  -4.90531517e-03 -1.81735007e-22j   4.90531580e-03 -7.83949703e-23j
  -9.81063032e-03 +7.55592711e-24j  -9.81063123e-03 +8.80009522e-24j
   9.81061824e-03 +5.08637304e-22j  -9.81064372e-03 +5.09008888e-22j
  -2.06980509e-10 +6.18321146e-14j  -1.60879250e-08 +2.33068950e-12j
  -2.21061012e-07 +5.85505099e-11j]
